In [1]:
import h5py
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import labrotation.file_handling as fh
import os

In [2]:
fpath_weights_seizures = fh.open_file("Select weights_seizures.hdf5 file!")
assert os.path.exists(fpath_weights_seizures)

In [5]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

dict_keys(['DATA_DOCU_FOLDER', 'DOWNLOADS_FOLDER', 'LOG_FOLDER', 'MATLAB_2P_FOLDER', 'FLUO_LV_MATCHED_FOLDER'])


In [6]:
fpath_out = env_dict["DOWNLOADS_FOLDER"]

In [4]:
dict_w_sz = dict()  # {mouse id: {injection type: str, all seizures: list[float], weight days: list[int], weights: list[float]}}
with h5py.File(fpath_weights_seizures, "r") as hf:
    for grp in hf:
        meta = hf[grp].attrs
        mouse_id = meta["id"]
        inj_type = meta["injection_type"]
        all_szs = hf[grp]["all_seizures"][()]  # in hours since injection
        w_days = hf[grp]["weight_days"][()]
        ws = hf[grp]["weights"][()]
        dict_w_sz[mouse_id] = {"injection_type": inj_type, "all_seizures": all_szs, "weight_days": w_days, "weights": ws}

In [ ]:
sz_times = []
mouse_ids = []
sz_colors = {}
#colors = ["0x000000", "0x444444", "0x808080", "0x8E8E8E", "0xA9A9A9", "0xCCCCCC", "0xEEEEEE"]
#colors = [(0, 0, 0), (80, 80, 80), (128, 128, 128), (150, 150, 150), (172, 172, 172), (212, 212, 212), (220, 220, 220)]
colors = [(0, 0, 0), (89, 89, 89), (115, 115, 115), (140, 140, 140), (166, 166, 166), (191, 191, 191), (217, 217, 217)]  # same colors as first draft of manuscript
colors = [(color[0]/255., color[1]/255., color[2]/255. ) for color in colors]
i_color = 0
for mouse_id in dict_w_sz:
    szs = dict_w_sz[mouse_id]["all_seizures"]
    if len(szs) > 0:
        sz_times.extend(szs)
        mouse_ids.extend([mouse_id]*len(szs))
        sz_colors[mouse_id] = colors[i_color]
        i_color += 1

In [ ]:
sz_times = [sz_time / 24. for sz_time in sz_times]  # convert to days p.i.

In [ ]:
df_sz = pd.DataFrame({"time": sz_times, "id": mouse_ids})

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sn.violinplot(data=df_sz, x="time", ax=ax, color="black", fill=False, inner=None)
#sn.stripplot(data=df_sz, x="time", hue="id", jitter=0.06, palette=sz_colors, size=10, alpha=0.8, ax=ax)
sn.swarmplot(data=df_sz, x="time", hue="id", palette=sz_colors, size=10, alpha=1.0, ax=ax)
ax.set_xlim((0, 8))
ax.set_ylabel("# of sz (k.d.e.)", fontsize=16)
ax.set_xlabel("d. p. i.", fontsize=16)
plt.savefig(os.path.join(fpath_out, "sz_kde.pdf"))
plt.show()

In [ ]:
# plot seizures per day
fig, ax = plt.subplots(figsize=(12, 6))
sn.histplot(
    df_sz,
    x="time", hue="id",
    multiple="dodge", #"stack",
    palette=sz_colors,
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
    bins=[i for i in range(8)],
    ax=ax
)
ax.set_ylabel("#/day", fontsize=16)
ax.set_xlabel("d. p. i.", fontsize=16)
plt.savefig(os.path.join(fpath_out, "sz_hist.pdf"))
plt.show()